In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn import metrics
from sklearn import preprocessing

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
from imblearn.over_sampling import SMOTE

In [16]:
from sklearn.utils import class_weight

In [17]:
final_df = pd.read_csv('final_df.csv')

In [18]:
final_df.drop(['Unnamed: 0', 'TeamName','TeamID'], axis=1, inplace=True)

In [19]:
final_df

,Season,Rank_WLK,Rank_DOL,Rank_COL,Rank_SAG,Rank_MOR,Rank_POM,Seed,Wins,Losses,...,Stl_pg,Stl_Ag_pg,Blk,Blk_Ag,Blk_pg,Blk_Ag_pg,PF,PF_Ag,PF_pg,PF_Ag_pg
0,2003,8,6,3,8,6,9,1,24,6,...,6.933,5.167,113.0,80.0,3.767,2.667,558.0,526.0,18.600,17.533
1,2003,9,4,8,9,20,12,2,24,5,...,6.414,7.379,127.0,94.0,4.379,3.241,536.0,639.0,18.483,22.034
2,2003,14,5,9,12,21,15,3,24,5,...,8.310,6.621,211.0,91.0,7.276,3.138,481.0,572.0,16.586,19.724
3,2003,6,16,6,6,14,14,4,24,6,...,7.200,5.767,142.0,73.0,4.733,2.433,680.0,678.0,22.667,22.600
4,2003,17,19,25,16,8,11,5,21,9,...,8.733,7.733,112.0,90.0,3.733,3.000,477.0,515.0,15.900,17.167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,2021,83,49,52,87,50,91,12,23,1,...,7.750,6.000,57.0,80.0,2.375,3.333,84.0,98.0,3.500,4.083
1193,2021,78,88,85,84,94,71,13,16,9,...,6.840,6.480,75.0,74.0,3.000,2.960,138.0,126.0,5.520,5.040
1194,2021,10,16,14,67,41,84,14,14,1,...,7.200,4.867,51.0,42.0,3.400,2.800,68.0,85.0,4.533,5.667
1195,2021,151,150,181,152,130,151,15,13,10,...,6.217,5.565,94.0,71.0,4.087,3.087,184.0,177.0,8.000,7.696


In [20]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()

In [21]:
cw = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(final_df['T_Wins']),y=final_df['T_Wins'])


In [22]:
categories=list(final_df['T_Wins'].sort_values().unique())
cw_list=list(cw)
weights=dict(zip(categories, cw))
weights

{0.0: 0.2923076923076923,
 1.0: 0.5294117647058824,
 2.0: 1.1875,
 3.0: 2.28,
 4.0: 5.181818181818182,
 5.0: 9.0,
 6.0: 9.5}

In [76]:
#### TRY train test split   KEEP THIS

X = final_df
scaled_X = scaler.fit_transform(X)
scaled_X = pd.DataFrame(scaled_X)
y = final_df['T_Wins']
#X,y = smote.fit_resample(X,y)
#scaled_X , y = sampler.fit_resample(X,y)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, stratify=y,test_size=0.3,random_state=101)
smote = SMOTE()#sampling_strategy={0:500, 1:500, 2:500, 3:500, 4:500, 5:500},random_state=101)
#smotenc=SMOTENC([61], random_state=101)
X_train, y_train = smote.fit_resample(X_train,y_train)

In [40]:
models = [RandomForestClassifier(n_estimators=100, class_weight=weights), KNeighborsClassifier(n_neighbors=17), svm.SVC(class_weight=weights), LogisticRegression(solver='lbfgs', class_weight=weights)]


In [77]:
model_selection_rank = pd.DataFrame()
for model in models:
    model.fit(X_train, y_train)
    scores = cross_val_score(model, X_train, y_train)
    model_selection_rank[model.__str__()] = scores
    model_selection_rank   
    y_predict=model.predict(X_test)
    ac=accuracy_score(y_predict,y_test)
    cm=confusion_matrix(y_test, y_predict)
    cr=classification_report(y_test, y_predict)
    print(model)
    print(f'Train Score: {model.score(X_train, y_train)}')
    print(f'Test Score: {model.score(X_test, y_test)}')
    print(f'Accuracy: {ac}')
    print(f'Confusion Matrix: \n {cm}')
    print(f'Classification Report: \n {cr}')

RandomForestClassifier(class_weight={0.0: 0.2923076923076923,
                                     1.0: 0.5294117647058824, 2.0: 1.1875,
                                     3.0: 2.28, 4.0: 5.181818181818182,
                                     5.0: 9.0, 6.0: 9.5})
Train Score: 1.0
Test Score: 0.7361111111111112
Accuracy: 0.7361111111111112
Confusion Matrix: 
 [[162  13   1   0   0   0   0]
 [ 31  54  11   1   0   0   0]
 [  6   9  24   4   0   0   0]
 [  2   1   6  14   0   0   0]
 [  1   1   2   3   3   0   0]
 [  0   0   0   2   1   3   0]
 [  0   0   0   0   0   0   5]]
Classification Report: 
               precision    recall  f1-score   support

         0.0       0.80      0.92      0.86       176
         1.0       0.69      0.56      0.62        97
         2.0       0.55      0.56      0.55        43
         3.0       0.58      0.61      0.60        23
         4.0       0.75      0.30      0.43        10
         5.0       1.00      0.50      0.67         6
         6.0  

In [39]:
##CHECK ONE  DELETE THIS IN FINAL VERSION
model_selection_rank = pd.DataFrame()

model=LogisticRegression(solver='lbfgs', class_weight=weights)
model.fit(X_train, y_train)

scores = cross_val_score(model, X_train, y_train)
model_selection_rank[model.__str__()] = scores
model_selection_rank   
y_predict=model.predict(X_test)
ac=accuracy_score(y_predict,y_test)
cm=confusion_matrix(y_test, y_predict)
cr=classification_report(y_test, y_predict)
print(model)
print(scores)
print(f'Train Score: {model.score(X_train, y_train)}')
print(f'Test Score: {model.score(X_test, y_test)}')
print(f'Accuracy: {ac}')
print(f'Confusion Matrix: \n {cm}')
print(f'Classification Report: \n {cr}')

LogisticRegression(class_weight={0.0: 0.2923076923076923,
                                 1.0: 0.5294117647058824, 2.0: 1.1875,
                                 3.0: 2.28, 4.0: 5.181818181818182, 5.0: 9.0,
                                 6.0: 9.5})
[0.37243402 0.37390029 0.34750733 0.36656891 0.38179148]
Train Score: 0.385450278674098
Test Score: 0.10837438423645321
Accuracy: 0.10837438423645321
Confusion Matrix: 
 [[18  4  0  1 35 40  0]
 [ 7  0  1  2 24 22  3]
 [ 1  0  0  2 11  6  2]
 [ 0  1  1  0  6  2  3]
 [ 0  0  0  0  2  1  2]
 [ 0  0  0  0  2  1  0]
 [ 0  0  0  0  2  0  1]]
Classification Report: 
               precision    recall  f1-score   support

         0.0       0.69      0.18      0.29        98
         1.0       0.00      0.00      0.00        59
         2.0       0.00      0.00      0.00        22
         3.0       0.00      0.00      0.00        13
         4.0       0.02      0.40      0.05         5
         5.0       0.01      0.33      0.03         3
      

In [86]:
#CHOSEN MODEL
model = RandomForestClassifier(n_estimators=100, class_weight=weights)

In [87]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
metrics.accuracy_score(y_test, y_pred)

0.7111111111111111

In [88]:
result = pd.DataFrame({'actual':y_test, 'predicted':y_pred})

In [89]:
result['actual points earned'] = round(2**(result['actual']-1)-.01)
result['predicted points earned'] = round(2**(result['predicted']-1)-.01)

In [90]:
result['difference in score'] = abs(result['actual points earned'] - result['predicted points earned'])

In [91]:
nums = [0,1,2,3,4,5,6]
dict={}
pred_list=[]
act_list=[]
pred_correct_list=[]
for num in nums:
   # result_pred = result[(result['difference in score'] == 0) & (result['actual'] == num)]['actual'].count()
    result_pred = result[result['predicted'] == num]['predicted'].count()
    result_act = result[result['actual'] == num]['actual'].count()
    result_pred_correct = result[(result['difference in score'] == 0) & (result['actual'] == num)]['actual'].count()
    pred_list.append(result_pred)
    act_list.append(result_act)
    pred_correct_list.append(result_pred_correct)
    #dict.update({f'pred{num}' : result_pred, f'act{num}' : result_act})
dict={'predicted': pred_list, 'actual': act_list, 'predicted correct': pred_correct_list}
pd.DataFrame.from_dict(dict)

,predicted,actual,predicted correct
0,195,176,162
1,88,97,55
2,33,43,14
3,30,23,14
4,6,10,3
5,3,6,3
6,5,5,5


In [92]:
result

,actual,predicted,actual points earned,predicted points earned,difference in score
468,0.0,0.0,0.0,0.0,0.0
555,3.0,3.0,4.0,4.0,0.0
89,0.0,0.0,0.0,0.0,0.0
446,0.0,2.0,0.0,2.0,2.0
963,2.0,2.0,2.0,2.0,0.0
...,...,...,...,...,...
121,2.0,1.0,2.0,1.0,1.0
5,1.0,0.0,1.0,0.0,1.0
10,0.0,0.0,0.0,0.0,0.0
1098,2.0,2.0,2.0,2.0,0.0


In [138]:
model_correct=[]
model_incorrect=[]
for num in list(result['actual'].sort_values().unique()):
    model_correct.append(result[(result['actual']==num) & (result['predicted']>=num)]['actual'].count())
    model_incorrect.append(result[(result['actual']==num) & (result['predicted']<num)]['actual'].count())

model_ac = pd.DataFrame()
model_ac['Round'] = [0,1,2,3,4,5,6]
model_ac['Model Correct'] = model_correct
model_ac['Model Incorrect'] = model_incorrect
model_ac.drop(0, inplace=True)
model_ac['Model Accuracy'] = round(model_ac['Model Correct']/(model_ac['Model Correct']+model_ac['Model Incorrect']),3)
model_ac


,Round,Model Correct,Model Incorrect,Model Accuracy
1,1,69,28,0.711
2,2,19,24,0.442
3,3,15,8,0.652
4,4,3,7,0.300
5,5,3,3,0.500
6,6,5,0,1.000


In [139]:
best_seed_correct = [23.1, 10.4, 4.2, 1.3, 0.6, 0.2]
best_seed_incorrect = [32-23.1, 16-10.4, 8-4.2, 4-1.3, 2-0.6, 1-0.2]
model_ac['Best-Seed Correct']=best_seed_correct
model_ac['Best-Seed Incorrect']=best_seed_incorrect
model_ac['Best-Seed Accuracy'] = round(model_ac['Best-Seed Correct']/(model_ac['Best-Seed Correct']+model_ac['Best-Seed Incorrect']),3)
model_ac


,Round,Model Correct,Model Incorrect,Model Accuracy,Best-Seed Correct,Best-Seed Incorrect,Best-Seed Accuracy
1,1,69,28,0.711,23.1,8.9,0.722
2,2,19,24,0.442,10.4,5.6,0.650
3,3,15,8,0.652,4.2,3.8,0.525
4,4,3,7,0.300,1.3,2.7,0.325
5,5,3,3,0.500,0.6,1.4,0.300
6,6,5,0,1.000,0.2,0.8,0.200


In [157]:
model_ac['Model Points'] = round(model_ac['Model Accuracy']*32,1)
model_ac['Best-Seed Points'] = round(model_ac['Best-Seed Accuracy']*32,1)

In [158]:
model_ac

,Round,Model Correct,Model Incorrect,Model Accuracy,Best-Seed Correct,Best-Seed Incorrect,Best-Seed Accuracy,Model Points,Best-Seed Points
1,1,69,28,0.711,23.1,8.9,0.722,22.8,23.1
2,2,19,24,0.442,10.4,5.6,0.650,14.1,20.8
3,3,15,8,0.652,4.2,3.8,0.525,20.9,16.8
4,4,3,7,0.300,1.3,2.7,0.325,9.6,10.4
5,5,3,3,0.500,0.6,1.4,0.300,16.0,9.6
6,6,5,0,1.000,0.2,0.8,0.200,32.0,6.4


In [159]:
model_ac['Model Points'].sum()

115.39999999999999

In [160]:
model_ac['Best-Seed Points'].sum()

87.10000000000001